### 1. Load libraries

In [239]:
# import packages
import numpy as np
import pandas as pd
import datetime as dt
import glob

### 2. Load data

In [213]:
# Download data from the period 2010-2020 
all_files = glob.glob("./data/datos*.csv")

dfs = []

for filename in all_files:
    df = pd.read_csv(filename, header=0, sep=';')
    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True)

In [214]:
df.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,V01,D02,...,D27,V27,D28,V28,D29,V29,D30,V30,D31,V31
0,28,79,4,1,28079004_1_38,2017,1,6.0,V,8.0,...,5.0,V,4.0,V,4.0,V,5.0,V,7.0,V
1,28,79,4,1,28079004_1_38,2017,2,8.0,V,4.0,...,4.0,V,4.0,V,0.0,N,0.0,N,0.0,N
2,28,79,4,1,28079004_1_38,2017,3,5.0,V,7.0,...,4.0,V,5.0,V,7.0,V,7.0,V,5.0,V
3,28,79,4,1,28079004_1_38,2017,4,4.0,V,4.0,...,3.0,V,4.0,V,4.0,V,4.0,V,0.0,N
4,28,79,4,1,28079004_1_38,2017,5,4.0,V,5.0,...,5.0,V,5.0,V,4.0,V,4.0,V,5.0,V


In [215]:
# Lower column names
df.columns = map(str.lower, df.columns)

### 3. Data cleansing

In [216]:
# Select columns with daily values
days= df.filter(regex=("^d.*$"))
days.head()

,d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,6.0,8.0,12.0,9.0,7.0,10.0,10.0,11.0,10.0,12.0,...,7.0,9.0,12.0,9.0,9.0,5.0,4.0,4.0,5.0,7.0
1,8.0,4.0,3.0,3.0,3.0,4.0,5.0,5.0,6.0,8.0,...,12.0,11.0,6.0,6.0,5.0,4.0,4.0,0.0,0.0,0.0
2,5.0,7.0,5.0,3.0,3.0,4.0,6.0,9.0,10.0,10.0,...,4.0,3.0,4.0,4.0,4.0,4.0,5.0,7.0,7.0,5.0
3,4.0,4.0,5.0,7.0,4.0,4.0,5.0,5.0,5.0,6.0,...,5.0,5.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,0.0
4,4.0,5.0,5.0,6.0,5.0,4.0,4.0,4.0,5.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [217]:
# Select columns with data validity
validity = df.filter(regex=("^v.*$"))
validity.head()

,v01,v02,v03,v04,v05,v06,v07,v08,v09,v10,...,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31
0,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,V
1,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,N,N,N
2,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,V
3,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,N
4,V,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,V


In [218]:
# Convert V in null values
validity = validity.replace('V', np.nan)

In [219]:
# Change column names in validty and use 'days' column names
validity.columns = days.columns

In [220]:
# Use combine_first to get valid data
valid_data= validity.combine_first(days)

In [222]:
# Replace 'N' values for np.nan
valid_data= valid_data.replace('N', np.nan)

valid_data.head()

,d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,6.0,8.0,12.0,9.0,7.0,10.0,10.0,11.0,10.0,12.0,...,7.0,9.0,12.0,9.0,9.0,5.0,4.0,4.0,5.0,7.0
1,8.0,4.0,3.0,3.0,3.0,4.0,5.0,5.0,6.0,8.0,...,12.0,11.0,6.0,6.0,5.0,4.0,4.0,NaN,NaN,NaN
2,5.0,7.0,5.0,3.0,3.0,4.0,6.0,9.0,10.0,10.0,...,4.0,3.0,4.0,4.0,4.0,4.0,5.0,7.0,7.0,5.0
3,4.0,4.0,5.0,7.0,4.0,4.0,5.0,5.0,5.0,6.0,...,5.0,5.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,NaN
4,4.0,5.0,5.0,6.0,5.0,4.0,4.0,4.0,5.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [223]:
# Create a new dataframe concatenating columns from df_match_no2 and valid measurements
df_valid = pd.concat([df[['estacion', 'magnitud', 'ano','mes']], valid_data], axis=1)

df_valid.head()

,estacion,magnitud,ano,mes,d01,d02,d03,d04,d05,d06,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,4,1,2017,1,6.0,8.0,12.0,9.0,7.0,10.0,...,7.0,9.0,12.0,9.0,9.0,5.0,4.0,4.0,5.0,7.0
1,4,1,2017,2,8.0,4.0,3.0,3.0,3.0,4.0,...,12.0,11.0,6.0,6.0,5.0,4.0,4.0,NaN,NaN,NaN
2,4,1,2017,3,5.0,7.0,5.0,3.0,3.0,4.0,...,4.0,3.0,4.0,4.0,4.0,4.0,5.0,7.0,7.0,5.0
3,4,1,2017,4,4.0,4.0,5.0,7.0,4.0,4.0,...,5.0,5.0,6.0,4.0,4.0,3.0,4.0,4.0,4.0,NaN
4,4,1,2017,5,4.0,5.0,5.0,6.0,5.0,4.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0


In [224]:
# Use melt to create a column 'day'
df_valid = pd.melt(df_valid,id_vars= ['estacion','magnitud','ano','mes'], value_vars =cols,
                             var_name ='dia', value_name ='medicion')
                                                           
df_valid.head()

,estacion,magnitud,ano,mes,dia,medicion
0,4,1,2017,1,d01,6.0
1,4,1,2017,2,d01,8.0
2,4,1,2017,3,d01,5.0
3,4,1,2017,4,d01,4.0
4,4,1,2017,5,d01,4.0


In [225]:
# Drop rows with NULL values
df_valid = df_valid.dropna()

In [226]:
# Remove 'd' from column 'day'
df_valid['dia'] = df_valid['dia'].str.replace('d', '')

In [227]:
# Create a new column 'date' joining year, month and day
df_valid['fecha'] = (df_valid['ano'].map(str) + "-" + df_valid['mes'].map(str) + "-" 
                                                             + df_valid['dia'].map(str))

In [228]:
# Convert 'date' to datetime
df_valid['fecha'] = pd.to_datetime(df_valid['fecha'], format='%Y-%m-%d')

df_valid.head()

,estacion,magnitud,ano,mes,dia,medicion,fecha
0,4,1,2017,1,01,6.0,2017-01-01
1,4,1,2017,2,01,8.0,2017-02-01
2,4,1,2017,3,01,5.0,2017-03-01
3,4,1,2017,4,01,4.0,2017-04-01
4,4,1,2017,5,01,4.0,2017-05-01


In [229]:
# Drop columns 'year','month','day'
df_valid.drop(columns=['ano', 'mes', 'dia'], inplace=True)

df_valid.head()

,estacion,magnitud,medicion,fecha
0,4,1,6.0,2017-01-01
1,4,1,8.0,2017-02-01
2,4,1,5.0,2017-03-01
3,4,1,4.0,2017-04-01
4,4,1,4.0,2017-05-01


In [230]:
# Use grouby to calculate daily average emission by date and particle.
df_avg_emissions = pd.DataFrame(df_valid.groupby(['fecha', 'magnitud'])
                                ['medicion']
                                .mean().reset_index())


In [231]:
# Use pivot to turn every particle in a column
df_avg_emissions = pd.pivot(df_avg_emissions, index= 'fecha', columns='magnitud',values='medicion')

In [233]:
df_avg_emissions.head()

magnitud,1,6,7,8,9,10,12,14,20,30,35,42,43,44
fecha,,,,,,,,,,,,,,
2010-01-01,7.9,0.322222,7.105263,20.315789,7.00,10.250,31.210526,60.583333,1.750,0.575,1.00,1.343333,1.133333,0.210000
2010-01-02,9.5,0.433333,20.684211,42.368421,10.00,15.250,74.000000,27.583333,3.675,1.000,1.10,1.416667,1.213333,0.206667
2010-01-03,11.6,0.555556,37.421053,54.052632,12.25,16.625,111.578947,12.583333,5.960,1.440,1.58,1.543333,1.366667,0.180000
2010-01-04,11.6,0.511111,31.736842,55.526316,9.00,9.750,104.052632,13.416667,5.100,1.280,1.86,1.483333,1.306667,0.180000
2010-01-05,11.9,0.422222,22.736842,48.368421,8.00,12.125,83.263158,23.166667,3.675,0.725,1.60,1.436667,1.256667,0.183333


In [234]:
# Create list with name of particles and rename columns
particles = ['so2', 'co','no','no2','pm2_5','pm10','nox','o3','tol','ben','ebe','tch','ch4','nmhc']

df_avg_emissions.columns = particles


In [236]:
# Convert index to datetime
df_avg_emissions.index = pd.to_datetime(df_avg_emissions.index)

In [238]:
df_avg_emissions.head()

,so2,co,no,no2,pm2_5,pm10,nox,o3,tol,ben,ebe,tch,ch4,nmhc
fecha,,,,,,,,,,,,,,
2010-01-01,7.9,0.322222,7.105263,20.315789,7.00,10.250,31.210526,60.583333,1.750,0.575,1.00,1.343333,1.133333,0.210000
2010-01-02,9.5,0.433333,20.684211,42.368421,10.00,15.250,74.000000,27.583333,3.675,1.000,1.10,1.416667,1.213333,0.206667
2010-01-03,11.6,0.555556,37.421053,54.052632,12.25,16.625,111.578947,12.583333,5.960,1.440,1.58,1.543333,1.366667,0.180000
2010-01-04,11.6,0.511111,31.736842,55.526316,9.00,9.750,104.052632,13.416667,5.100,1.280,1.86,1.483333,1.306667,0.180000
2010-01-05,11.9,0.422222,22.736842,48.368421,8.00,12.125,83.263158,23.166667,3.675,0.725,1.60,1.436667,1.256667,0.183333
